In [4]:
# ============================================
# 1. IMPORTAR BIBLIOTECAS NECESSÁRIAS
# ============================================

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import math

# Configurar matplotlib para bom display no notebook
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# ============================================
# 2. CONSTANTES DO MERCADO PORTUGUÊS
# ============================================

# Taxas Euribor atuais (exemplo - atualizar conforme necessário)
EURIBOR_RATES = {
    '3 meses': 3.85,
    '6 meses': 3.75,
    '12 meses': 3.55
}

# Taxa de juro fixa de referência (para demonstração)
TAXA_JURO_FIXA_REF = 4.5

# Tabelas de IMT 2024
IMT_TABELAS = {
    'urbano_residencial': [
        (0, 101917, 0, 0),
        (101917, 158165, 0.02, 0),
        (158165, 237456, 0.05, 4780),
        (237456, 564989, 0.07, 9323),
        (564989, 1000000, 0.08, 14911),
        (1000000, float('inf'), 0.06, 0)  # Para casas > 1M€
    ],
    'urbano_residencial_segunda': [
        (0, 101917, 0.01, 0),
        (101917, 158165, 0.02, 1019),
        (158165, 237456, 0.05, 2573),
        (237456, 564989, 0.07, 6338),
        (564989, 1000000, 0.08, 11883),
        (1000000, float('inf'), 0.06, 0)
    ]
}

# Imposto do Selo
STAMP_DUTY_RATE = 0.0008  # 0,08% sobre o valor do crédito

# Custos de processo
CUSTOS_NOTARIADO = 800  # € (estimado)
CUSTOS_REGISTRO = 250   # € (estimado)
CUSTOS_AVALIACAO = 400  # € (estimado)

# Limites DSTI (Decisão do Banco de Portugal)
DSTI_LIMITE_PADRAO = 0.30  # 30% do rendimento líquido
DSTI_LIMITE_ELEVADO = 0.40  # 40% para clientes de risco mais baixo

# ============================================
# 3. FUNÇÕES DE CÁLCULO
# ============================================

def calcular_imt(valor_imovel, tipo='urbano_residencial', primeira_casa=True):
    """
    Calcula o IMT conforme tabelas legais portuguesas
    """
    if not primeira_casa:
        tabela = IMT_TABELAS['urbano_residencial_segunda']
    else:
        tabela = IMT_TABELAS[tipo]
    
    for min_val, max_val, taxa, deducao in tabela:
        if min_val <= valor_imovel < max_val:
            if taxa == 0.06 and valor_imovel > 1000000:  # Caso especial > 1M€
                return valor_imovel * taxa
            return max(0, valor_imovel * taxa - deducao)
    
    return valor_imovel * 0.08

def calcular_selo(valor_contrato):
    """Calcula o Imposto do Selo sobre o contrato de crédito"""
    return valor_contrato * STAMP_DUTY_RATE

def calcular_pagamento_mensal(principal, taxa_anual, meses, frequencia='mensal'):
    """
    Calcula o pagamento mensal considerando diferentes frequências de capitalização
    """
    if taxa_anual == 0:
        return principal / meses
    
    # Converter taxa anual para decimal
    taxa_anual = taxa_anual / 100
    
    if frequencia == 'mensal':
        taxa_periodica = taxa_anual / 12
        num_pagamentos = meses
    elif frequencia == 'semanal':
        taxa_periodica = taxa_anual / 52
        num_pagamentos = meses * 4.33  # Aproximação
    elif frequencia == 'diaria':
        taxa_periodica = taxa_anual / 365
        num_pagamentos = meses * 30.44  # Aproximação
    elif frequencia == 'continua':
        # Para capitalização contínua: usar fórmula ajustada
        # pagamento = P * (e^r - 1) / (1 - e^(-r*t))
        taxa_mensal_efetiva = np.exp(taxa_anual / 12) - 1
        numerador = principal * taxa_mensal_efetiva
        denominador = 1 - (1 + taxa_mensal_efetiva)**(-meses)
        return numerador / denominador
    else:
        raise ValueError("Frequência inválida")
    
    # Fórmula padrão do empréstimo
    denominador = 1 - (1 + taxa_periodica)**(-num_pagamentos)
    if denominador == 0:
        return principal / meses
    
    return principal * taxa_periodica / denominador

def calcular_tempo_pagamento(principal, taxa_anual, pagamento_mensal, frequencia='mensal'):
    """
    Calcula quantos meses são necessários para pagar o empréstimo
    com um pagamento mensal específico
    """
    if taxa_anual <= 0 or pagamento_mensal <= 0:
        return float('inf')
    
    taxa_anual = taxa_anual / 100
    
    if frequencia == 'mensal':
        taxa_periodica = taxa_anual / 12
    elif frequencia == 'semanal':
        taxa_periodica = taxa_anual / 52
    elif frequencia == 'diaria':
        taxa_periodica = taxa_anual / 365
    elif frequencia == 'continua':
        taxa_mensal_efetiva = np.exp(taxa_anual / 12) - 1
        taxa_periodica = taxa_mensal_efetiva
    
    # Número de pagamentos necessários
    if pagamento_mensal <= principal * taxa_periodica:
        return float('inf')  # Nunca pagará o empréstimo
    
    numerador = -np.log(1 - (principal * taxa_periodica) / pagamento_mensal)
    denominador = np.log(1 + taxa_periodica)
    
    return numerador / denominador

def gerar_amortizacao(principal, taxa_anual, meses, frequencia='mensal'):
    """
    Gera quadro de amortização completo
    """
    pagamento_mensal = calcular_pagamento_mensal(principal, taxa_anual, meses, frequencia)
    
    taxa_anual = taxa_anual / 100
    
    if frequencia == 'mensal':
        taxa_periodica = taxa_anual / 12
    elif frequencia == 'semanal':
        taxa_periodica = taxa_anual / 52
    elif frequencia == 'diaria':
        taxa_periodica = taxa_anual / 365
    elif frequencia == 'continua':
        taxa_periodica = np.exp(taxa_anual / 12) - 1
    
    meses = int(meses)
    saldo = principal
    dados = []
    
    for mes in range(1, meses + 1):
        juros = saldo * taxa_periodica
        principal_pago = min(pagamento_mensal - juros, saldo)
        saldo -= principal_pago
        
        dados.append({
            'Mês': mes,
            'Data': datetime.now().replace(day=1) + timedelta(days=30*mes),
            'Pagamento': round(pagamento_mensal, 2),
            'Juros': round(juros, 2),
            'Principal': round(principal_pago, 2),
            'Saldo': round(max(0, saldo), 2)
        })
        
        if saldo <= 0:
            break
    
    return pd.DataFrame(dados)

def calcular_seguros(valor_imovel, valor_emprestimo, idade):
    """
    Estima custos de seguros obrigatórios
    """
    # Seguro vida (aproximação: 0,05% por ano do valor do empréstimo)
    seguro_vida_anual = valor_emprestimo * 0.0005
    
    # Seguro multi-riscos (aproximação: 0,15% do valor do imóvel)
    seguro_habitacao_anual = valor_imovel * 0.0015
    
    # Ajuste por idade (risco maior > 50 anos)
    if idade > 50:
        seguro_vida_anual *= 1.5
    
    return {
        'seguro_vida_mensal': seguro_vida_anual / 12,
        'seguro_habitacao_mensal': seguro_habitacao_anual / 12,
        'total_anual': seguro_vida_anual + seguro_habitacao_anual,
        'total_mensal': (seguro_vida_anual + seguro_habitacao_anual) / 12
    }

def validar_dsti(rendimento_liquido, pagamento_total, limite=DSTI_LIMITE_PADRAO):
    """
    Valida o cumprimento do limite DSTI do Banco de Portugal
    """
    dsti = pagamento_total / rendimento_liquido
    return {
        'dsti': dsti,
        'valido': dsti <= limite,
        'limite': limite,
        'margem': limite - dsti,
        'limite_utilizado': (dsti / limite) * 100,
        'mensagem': f"DSTI: {dsti:.1%} {'✓ Válido' if dsti <= limite else '✗ Excedido'}"
    }

# ============================================
# 4. INTERFACE GRÁFICA COM IPYWIDGETS
# ============================================

def criar_interface():
    """Cria a interface interativa da calculadora"""
    
    # Título
    titulo = widgets.HTML(
    value="<h1>🏠 Calculadora de Crédito Habitação Portugal</h1>"
          "<p style='color: #666;'>Calculadora conforme regulamentação Banco de Portugal</p>",
    layout=widgets.Layout(margin='0 0 20px 0')
    )
    
    # Agrupar widgets por seções
    
    # Seção 1: Características do Imóvel
    secao_imovel = widgets.HTML("<h3>🏢 Características do Imóvel</h3>")
    
    valor_imovel = widgets.BoundedFloatText(
        value=250000,
        min=50000,
        max=5000000,
        step=5000,
        description='Valor do Imóvel (€):',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='400px')
    )
    
    primeira_casa = widgets.Checkbox(
    value=True,
    description='Primeira habitação própria e permanente',
    style={'description_width': '350px'},
    layout=widgets.Layout(width='400px')
    )
    
    localizacao = widgets.Dropdown(
        options=['Lisboa/Porto', 'Centro', 'Norte (exceto Porto)', 'Alentejo', 'Algarve', 'Açores/Madeira'],
        value='Centro',
        description='Localização:',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='400px')
    )
    
    # Seção 2: Características do Crédito
    secao_credito = widgets.HTML("<h3>💰 Características do Crédito</h3>")
    
    tipo_taxa = widgets.RadioButtons(
        options=[('Taxa Variável (Euribor)', 'variavel'), 
                ('Taxa Fixa', 'fixa')],
        value='variavel',
        description='Tipo de Taxa:',
        style={'description_width': '200px'}
    )
    
    euribor_opcao = widgets.Dropdown(
        options=[('Euribor 3 meses', '3 meses'), 
                ('Euribor 6 meses', '6 meses'), 
                ('Euribor 12 meses', '12 meses')],
        value='6 meses',
        description='Euribor:',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='400px')
    )
    
    spread = widgets.BoundedFloatText(
        value=1.5,
        min=0.5,
        max=5.0,
        step=0.1,
        description='Spread Bancário (%):',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='400px')
    )
    
    # Atualizar taxa quando Euribor ou spread mudam
    def atualizar_taxa(*args):
        if tipo_taxa.value == 'variavel':
            euribor_valor = EURIBOR_RATES[euribor_opcao.value]
            taxa_juro_fixa.value = round(euribor_valor + spread.value, 2)
        else:
            taxa_juro_fixa.value = TAXA_JURO_FIXA_REF
    
    euribor_opcao.observe(atualizar_taxa, 'value')
    spread.observe(atualizar_taxa, 'value')
    tipo_taxa.observe(atualizar_taxa, 'value')
    
    taxa_juro_fixa = widgets.BoundedFloatText(
        value=round(EURIBOR_RATES['6 meses'] + 1.5, 2),
        min=0.1,
        max=15.0,
        step=0.01,
        description='Taxa de Juro Anual (%):',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='400px')
    )
    
    prazo_anos = widgets.BoundedIntText(
        value=30,
        min=5,
        max=40,
        step=1,
        description='Prazo (anos):',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='400px')
    )
    
    entrada_percentagem = widgets.BoundedFloatText(
        value=20,
        min=5,
        max=50,
        step=5,
        description='Entrada (%):',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='400px')
    )
    
    frequencia_capitalizacao = widgets.Dropdown(
        options=[('Mensal', 'mensal'), 
                ('Semanal', 'semanal'), 
                ('Diária', 'diaria'), 
                ('Contínua', 'continua')],
        value='mensal',
        description='Capitalização:',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='400px')
    )
    
    # Seção 3: Perfil do Cliente
    secao_cliente = widgets.HTML("<h3>👤 Perfil do Cliente</h3>")
    
    rendimento_liquido = widgets.BoundedFloatText(
        value=1500,
        min=500,
        max=10000,
        step=100,
        description='Rendimento Líquido Mensal (€):',
        style={'description_width': '200px'},
        layout=widgets.Layout(width='400px')
    )
    
    idade = widgets.BoundedIntText(
    value=35,
    min=18,
    max=80,
    step=1,
    description='Idade:',
    style={'description_width': '200px'},
    layout=widgets.Layout(width='400px')
    )
    
    # Seção 4: Botão de cálculo
    botao_calcular = widgets.Button(
        description='Calcular Crédito Habitação',
        button_style='success',
        icon='calculator',
        layout=widgets.Layout(width='300px', height='50px', margin='20px 0')
    )
    
    # Saída para resultados
    saida_resultados = widgets.Output()
    
    # ============================================
    # 5. FUNÇÃO DE CÁLCULO PRINCIPAL
    # ============================================
    
    def calcular_credito(btn):
        with saida_resultados:
            saida_resultados.clear_output()
            
            # Calcular valores principais
            valor_imovel_val = valor_imovel.value
            entrada_percent = entrada_percentagem.value / 100
            entrada_valor = valor_imovel_val * entrada_percent
            valor_emprestimo = valor_imovel_val - entrada_valor
            
            if valor_emprestimo <= 0:
                print("❌ O valor da entrada deve ser inferior ao valor do imóvel!")
                return
            
            # Calcular pagamento mensal
            meses = prazo_anos.value * 12
            taxa = taxa_juro_fixa.value
            
            pagamento_mensal = calcular_pagamento_mensal(
                valor_emprestimo, 
                taxa, 
                meses, 
                frequencia_capitalizacao.value
            )
            
            # Calcular seguros
            seguros = calcular_seguros(valor_imovel_val, valor_emprestimo, idade.value)
            
            # Calcular IMT
            imt = calcular_imt(valor_imovel_val, primeira_casa=primeira_casa.value)
            
            # Calcular selo
            selo = calcular_selo(valor_emprestimo)
            
            # Custos totais de aquisição
            custos_processo = CUSTOS_NOTARIADO + CUSTOS_REGISTRO + CUSTOS_AVALIACAO
            custos_totais = imt + selo + custos_processo + entrada_valor
            
            # Pagamento total mensal (crédito + seguros)
            pagamento_total_mensal = pagamento_mensal + seguros['total_mensal']
            
            # Validar DSTI
            dsti_info = validar_dsti(rendimento_liquido.value, pagamento_total_mensal)
            
            # Gerar quadro de amortização
            amortizacao = gerar_amortizacao(valor_emprestimo, taxa, meses, frequencia_capitalizacao.value)
            
            # Total pago ao longo do empréstimo
            total_pago_credito = pagamento_mensal * len(amortizacao)
            total_juros = total_pago_credito - valor_emprestimo
            
            # Criar interface de resultados
            display(HTML(f"""
            <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                        color: white; padding: 20px; border-radius: 10px; margin: 20px 0;">
                <h2 style="margin-top: 0;">💡 Resultados do Financiamento</h2>
                <p style="font-size: 1.1em;">Data da simulação: {datetime.now().strftime('%d/%m/%Y')}</p>
            </div>
            """))
            
            # ============================================
            # 6. TABELA RESUMO
            # ============================================
            
            resumo = pd.DataFrame({
                'Descrição': [
                    'Valor do Imóvel',
                    'Entrada (down payment)',
                    'Montante Financiado',
                    'Taxa de Juro Nominal Anual (TAN)',
                    'Prazo do Financiamento',
                    'Pagamento Mensal (crédito)',
                    'Seguro Vida (mensal)',
                    'Seguro Habitação (mensal)',
                    'PAGAMENTO TOTAL MENSAL',
                    'DSTI (Rácio de Esforço)',
                    'IMT',
                    'Imposto do Selo',
                    'Custos de Processo',
                    'CUSTO TOTAL INICIAL'
                ],
                'Valor': [
                    f"{valor_imovel_val:,.2f} €",
                    f"{entrada_valor:,.2f} €",
                    f"{valor_emprestimo:,.2f} €",
                    f"{taxa:.3f}%",
                    f"{prazo_anos.value} anos ({meses} meses)",
                    f"{pagamento_mensal:,.2f} €",
                    f"{seguros['seguro_vida_mensal']:,.2f} €",
                    f"{seguros['seguro_habitacao_mensal']:,.2f} €",
                    f"<strong style='color: red;'>{pagamento_total_mensal:,.2f} €</strong>",
                    f"<strong style='color: {'green' if dsti_info['valido'] else 'red'};'>{dsti_info['dsti']:.2%}</strong>",
                    f"{imt:,.2f} €",
                    f"{selo:,.2f} €",
                    f"{custos_processo:,.2f} €",
                    f"<strong>{custos_totais:,.2f} €</strong>"
                ]
            })
            
            display(HTML("<h3>📊 Resumo Financeiro</h3>"))
            display(resumo)
            
            # ============================================
            # 7. ANÁLISE DSTI
            # ============================================
            
            display(HTML(f"<h3>🏦 Análise Rácio de Esforço (DSTI)</h3>"))
            dsti_color = 'green' if dsti_info['valido'] else 'red'
            display(HTML(f"""
            <div style="border: 2px solid #{dsti_color}; padding: 15px; border-radius: 5px; margin: 10px 0;">
                <p><strong>Rácio de Esforço Calculado:</strong> <span style="color: {dsti_color}; font-size: 1.3em;">{dsti_info['dsti']:.2%}</span></p>
                <p><strong>Limite Permitido:</strong> {dsti_info['limite']:.0%}</p>
                <p><strong>Utilização do Limite:</strong> {dsti_info['limite_utilizado']:.1f}%</p>
                <p><strong>Status:</strong> <span style="color: {dsti_color};">{dsti_info['mensagem']}</span></p>
                <hr>
                <p style="font-size: 0.9em; color: #666;">
                <strong>Nota:</strong> De acordo com o Banco de Portugal, o limite de esforço raramente pode exceder 30% do rendimento líquido mensal (40% para clientes de risco mais baixo).
                </p>
            </div>
            """))
            
            # ============================================
            # 8. QUADRO DE AMORTIZAÇÃO
            # ============================================
            
            display(HTML("<h3>📈 Quadro de Amortização (Primeiros 12 Meses)</h3>"))
            display(amortizacao.head(12))
            
            # ============================================
            # 9. VISUALIZAÇÕES
            # ============================================
            
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
            fig.suptitle('Análise Gráfica do Financiamento', fontsize=16, fontweight='bold')
            
            # Gráfico 1: Evolução do Saldo
            ax1.plot(amortizacao['Mês'], amortizacao['Saldo'], 
                    color='#2ecc71', linewidth=2, marker='o', markersize=3)
            ax1.set_title('Evolução do Saldo Devedor', fontweight='bold')
            ax1.set_xlabel('Mês')
            ax1.set_ylabel('Saldo (€)')
            ax1.grid(True, alpha=0.3)
            
            # Gráfico 2: Distribuição Juros vs Principal
            cores = ['#e74c3c', '#3498db']
            total_juros_graf = amortizacao['Juros'].sum()
            total_principal_graf = amortizacao['Principal'].sum()
            ax2.pie([total_juros_graf, total_principal_graf], 
                   labels=['Juros', 'Principal'], 
                   colors=cores, autopct='%1.1f%%', startangle=90)
            ax2.set_title('Distribuição Total de Pagamentos', fontweight='bold')
            
            # Gráfico 3: Juros vs Principal ao longo do tempo
            ax3.stackplot(amortizacao['Mês'][:60], 
                         amortizacao['Juros'][:60], 
                         amortizacao['Principal'][:60],
                         labels=['Juros', 'Principal'],
                         colors=['#e74c3c', '#3498db'], alpha=0.8)
            ax3.set_title('Composição dos Pagamentos (5 primeiros anos)', fontweight='bold')
            ax3.set_xlabel('Mês')
            ax3.set_ylabel('Valor (€)')
            ax3.legend(loc='upper right')
            ax3.grid(True, alpha=0.3)
            
            # Gráfico 4: Comparação de cenários
            prazos = [15, 20, 25, 30, 35]
            pagamentos = [calcular_pagamento_mensal(valor_emprestimo, taxa, p*12, frequencia_capitalizacao.value) 
                         for p in prazos]
            ax4.plot(prazos, pagamentos, 'o-', color='#9b59b6', linewidth=3, markersize=8)
            ax4.axhline(y=rendimento_liquido.value * DSTI_LIMITE_PADRAO, 
                       color='red', linestyle='--', 
                       label=f'Limite DSTI ({DSTI_LIMITE_PADRAO:.0%})')
            ax4.set_title('Pagamento Mensal vs Prazo', fontweight='bold')
            ax4.set_xlabel('Prazo (anos)')
            ax4.set_ylabel('Pagamento Mensal (€)')
            ax4.legend()
            ax4.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # ============================================
            # 10. INFORMAÇÕES ADICIONAIS
            # ============================================
            
            display(HTML(f"""
            <div style="background-color: #f8f9fa; padding: 20px; border-radius: 5px; margin-top: 20px;">
                <h3>📋 Informações Complementares</h3>
                <ul>
                    <li><strong>Total Pago ao Banco:</strong> {total_pago_credito:,.2f} €</li>
                    <li><strong>Total de Juros:</strong> {total_juros:,.2f} € ({(total_juros/valor_emprestimo)*100:.1f}% do valor financiado)</li>
                    <li><strong>Custo Efetivo Total (estimado):</strong> {taxa + 0.5:.3f}% (inclui taxas bancárias)</li>
                    <li><strong>Valor do Imóvel após {prazo_anos.value} anos (estimado):</strong> {valor_imovel_val * (1.02**prazo_anos.value):,.2f} € (assumindo 2% valorização anual)</li>
                </ul>
                
                <h4>📝 Requisitos para Formalização:</h4>
                <ul>
                    <li>✓ Avaliação bancária do imóvel</li>
                    <li>✓ Seguro de vida e seguro multi-riscos habitacionais</li>
                    <li>✓ Abertura de conta de depósitos à ordem</li>
                    <li>✓ Comprovativos de rendimentos (3 últimos recibos)</li>
                    <li>✓ Declaração de IRS (últimos 2 anos)</li>
                    <li>✓ Mapa de responsabilidades de crédito</li>
                </ul>
                
                <p style="font-size: 0.9em; color: #666; margin-top: 15px;">
                <strong>Aviso Legal:</strong> Esta calculadora serve apenas para fins informativos. 
                Os valores apresentados não constituem uma proposta vinculativa. Para uma simulação oficial, 
                contacte a sua instituição bancária. As taxas Euribor e spreads são exemplificativos e devem 
                ser atualizados conforme as condições de mercado atuais.
                </p>
            </div>
            """))
    
    botao_calcular.on_click(calcular_credito)
    
    # Montar interface
    interface = widgets.VBox([
        titulo,
        secao_imovel,
        valor_imovel,
        primeira_casa,
        localizacao,
        widgets.HTML("<hr>"),
        secao_credito,
        tipo_taxa,
        euribor_opcao,
        spread,
        taxa_juro_fixa,
        prazo_anos,
        entrada_percentagem,
        frequencia_capitalizacao,
        widgets.HTML("<hr>"),
        secao_cliente,
        rendimento_liquido,
        idade,
        widgets.HTML("<hr>"),
        widgets.HBox([botao_calcular], layout=widgets.Layout(justify_content='center')),
        saida_resultados
    ])
    
    return interface

# ============================================
# 11. EXECUTAR APLICAÇÃO
# ============================================

# Criar e exibir a interface
calculadora = criar_interface()
display(calculadora)